Примечание: для данного датасета запись в таблице будет определяться как "арендное жилье"

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/seattle.csv')

In [3]:
df

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,latitude,longitude,location,name,currency,rate_type
0,0,2318,2536,Entire home/apt,"Seattle, WA, United States",21,5.0,8,4.0,2.5,250,2018-12-20 03:46:14.698548,47.610819,-122.290816,0101000020E6100000D449B6BA9C925EC0416326512FCE...,"Casa Madrona - Urban Oasis, 1 block from the P...",USD,nightly
1,1,3335,4193,Entire home/apt,"Seattle, WA, United States",1,NaN,4,2.0,1.0,100,2018-12-20 04:08:45.277643,47.529846,-122.275840,0101000020E61000006FBBD05CA7915EC04DF564FED1C3...,Sweet Seattle Urban Homestead 2 Bdr,USD,nightly
2,2,4291,35749,Private room,"Seattle, WA, United States",63,4.5,2,1.0,1.0,82,2018-12-20 03:04:19.861014,47.687801,-122.313427,0101000020E6100000BDAB1E300F945EC0FB93F8DC09D8...,Sunrise in Seattle Master Suite,USD,nightly
3,3,5682,8993,Entire home/apt,"Seattle, WA, United States",462,5.0,2,0.0,1.0,49,2018-12-20 04:11:25.482975,47.523980,-122.359891,0101000020E6100000FCC7427408975EC009E1D1C611C3...,"Cozy Studio, min. to downtown -WiFi",USD,nightly
4,4,6606,14942,Entire home/apt,"Seattle, WA, United States",134,4.5,2,1.0,1.0,90,2018-12-20 03:12:38.391711,47.654109,-122.337605,0101000020E6100000D47D00529B955EC07782FDD7B9D3...,"Fab, private seattle urban cottage!",USD,nightly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7571,7970,30976187,231435350,Entire home/apt,"Seattle, WA, United States",0,NaN,4,1.0,1.0,78,2018-12-20 03:56:11.441877,47.611444,-122.330790,0101000020E610000001F6D1A92B955EC0EFAA07CC43CE...,Lovely Downtown Condo WS 97 (Includes Parking),USD,nightly
7572,7971,30982022,215426444,Private room,"Seattle, WA, United States",0,NaN,3,1.0,1.0,34,2018-12-20 03:42:51.04026,47.575621,-122.297958,0101000020E610000038656EBE11935EC006F2ECF2ADC9...,2 Beds&Bath Free Parking& LR STA Airport DT UW...,USD,nightly
7573,7972,30994069,230457966,Entire home/apt,"Seattle, WA, United States",0,NaN,2,1.0,1.0,80,2018-12-20 03:19:17.810312,47.621749,-122.326110,0101000020E610000082397AFCDE945EC00DA9A27895CF...,Vintage Private Courtyard Apartment,USD,nightly
7574,7973,30998202,48005494,Entire home/apt,"Washington, WA, United States",0,NaN,4,2.0,2.0,129,2018-12-20 03:26:52.53791,47.624673,-122.336345,0101000020E610000014CB2DAD86955EC0DCD8EC48F5CF...,2BR Apartment,USD,nightly


#### 1. Сгруппируйте арендное жилье по параметру room_type и выведите количество в каждом типе.

In [4]:
df.groupby(by=df.room_type).room_id.count()

room_type
Entire home/apt    5603
Private room       1844
Shared room         129
Name: room_id, dtype: int64

#### 2. Посчитайте среднее количество спален (bedrooms) и ванн (bathrooms)  (каждого по отдельности), где аренда (price) < 90 у.е.

In [5]:
df[df.price < 90][['bedrooms', 'bathrooms']].mean()

bedrooms     0.957565
bathrooms    1.124463
dtype: float64

#### 3. Выведите топ-5 апартаментов (room_type = "Entire home/apt") с наибольшим количеством отзывов (reviews)

In [6]:
df[df.room_type == "Entire home/apt"].sort_values(by='reviews', ascending=False).head()

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,latitude,longitude,location,name,currency,rate_type
28,29,25002,102684,Entire home/apt,"Seattle, WA, United States",687,5.0,4,1.0,1.0,70,2018-12-20 02:58:36.967441,47.679412,-122.370540,0101000020E6100000087767EDB6975EC04641F0F8F6D6...,"Beautiful, Private Spot in Quiet Ballard",USD,nightly
702,734,3861673,19928221,Entire home/apt,"Seattle, WA, United States",644,5.0,2,1.0,1.0,45,2018-12-20 02:52:55.955789,47.684030,-122.374086,0101000020E61000000EF96706F1975EC0CFBD874B8ED7...,Beautiful Guest Suite for 2 in Ballard,USD,nightly
22,22,20927,79450,Entire home/apt,"Seattle, WA, United States",489,5.0,2,1.0,1.0,59,2018-12-20 02:58:36.953496,47.673339,-122.395120,0101000020E6100000B77F65A549995EC0880FECF82FD6...,Cottage in the Heart of Ballard,USD,nightly
511,536,2686374,2231298,Entire home/apt,"Seattle, WA, United States",488,5.0,6,2.0,1.5,125,2018-12-20 03:05:39.14014,47.671250,-122.382639,0101000020E610000025CB49287D985EC052B81E85EBD5...,Bunkhouse 2nd floor & Airstream-No cleaning fees,USD,nightly
3,3,5682,8993,Entire home/apt,"Seattle, WA, United States",462,5.0,2,0.0,1.0,49,2018-12-20 04:11:25.482975,47.523980,-122.359891,0101000020E6100000FCC7427408975EC009E1D1C611C3...,"Cozy Studio, min. to downtown -WiFi",USD,nightly


#### 4. Найдите хозяина (host_id) с максимальным числом сдаваемого в аренду жилья

In [7]:
df.groupby(by=df.host_id).host_id.count().sort_values(ascending=False).head(1) 

host_id
8534462    321
Name: host_id, dtype: int64

#### 5. Найдите хозяина, у которого наибольшее число отзывов, рейтинг (overall_satisfaction) которых > 4

In [8]:
df[df.overall_satisfaction > 4].sort_values(by='reviews', ascending=False).host_id.head(1)

28    102684
Name: host_id, dtype: int64

#### 6. Найдите минимальную и максимальную стоимость видов жилья, где число проживающих (accommodates) варьирует от 2 до 6 человек включительно

In [9]:
pd.pivot_table(df[(df.accommodates >= 2) & (df.accommodates <= 6)],
               index=df.room_type, 
               values='price', 
               aggfunc=[max, min])

,max,min
,price,price
room_type,,
Entire home/apt,960,21
Private room,999,16
Shared room,180,15


#### 7. Посчитайте количество комнат (ванна + спальня) для каждой записи (нужно добавить новую колонку total_rooms). Найдите максимальное значение total_rooms для каждой из категорий (room_type) арендного жилья.

In [10]:
df['total_rooms'] = df.bedrooms + df.bathrooms

pd.pivot_table(df,
               index=df.room_type,
               values='total_rooms',
               aggfunc=max)

,total_rooms
room_type,
Entire home/apt,14.0
Private room,8.0
Shared room,9.0


#### 8. Посчитайте процентное соотношение числа отзывов по следующим категориям: 
* overall_satisfaction >= 0 and overall_satisfaction < 1
* overall_satisfaction >= 1 and overall_satisfaction < 2 
* overall_satisfaction >= 2 and overall_satisfaction < 3
* overall_satisfaction >= 3 and overall_satisfaction < 4
* overall_satisfaction >= 4 and overall_satisfaction < 5
* overall_satisfaction == 5

In [11]:
df.overall_satisfaction.dropna().astype(int).value_counts(normalize=True)

5    0.722595
4    0.269376
3    0.007865
2    0.000164
Name: overall_satisfaction, dtype: float64

#### 9. В какой час было произведено наибольшее число изменений записи (last_modified)? 


In [12]:
df.groupby(pd.to_datetime(df.last_modified).dt.hour)\
  .count()\
  .sort_values(by='Unnamed: 0', ascending=False)\
  .last_modified\
  .head(1)

last_modified
3    5409
Name: last_modified, dtype: int64

#### 10. Найдите топ-10 хозяев, которые заработали больше всех денег и топ-10, которые заработали меньше всего.

In [13]:
temp_df = df.sort_values(by='price', ascending=False)
temp_df.head(10)

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,latitude,longitude,location,name,currency,rate_type,total_rooms
71,73,152568,734114,Entire home/apt,"Mercer Island, WA, United States",2,NaN,10,3.0,4.5,5900,2018-12-20 03:39:23.645473,47.592590,-122.246703,0101000020E6100000CE3461FBC98F5EC0E4F736FDD9CB...,Luxury Waterfront Mansion,USD,nightly,7.5
3994,4187,20178074,15178427,Entire home/apt,"Seattle, WA, United States",0,NaN,12,6.0,6.0,3000,2018-12-20 03:29:42.272486,47.658215,-122.381796,0101000020E6100000976F7D586F985EC04A5E9D6340D4...,Mega yacht,USD,nightly,12.0
5691,5956,24885575,117796167,Entire home/apt,"Kirkland, WA, United States",2,NaN,8,3.0,3.5,2800,2018-12-20 02:57:04.779652,47.707262,-122.248111,0101000020E6100000C8B1F50CE18F5EC00EDAAB8F87DA...,Waterfront luxury - your own private resort!,USD,nightly,6.5
4260,4475,21015000,67467952,Entire home/apt,"Seattle, WA, United States",3,5.0,16,8.0,6.0,1650,2018-12-20 03:33:18.975047,47.630824,-122.368964,0101000020E6100000B0592E1B9D975EC015C440D7BED0...,Spacious Family-Friendly Tri-Plex with Views i...,USD,nightly,14.0
3069,3220,16816051,24047404,Entire home/apt,"Seattle, WA, United States",2,NaN,15,5.0,3.5,1395,2018-12-20 03:40:55.660377,47.547348,-122.266204,0101000020E61000001F84807C09915EC0F8C3CF7F0FC6...,Seattle Executive Home for Large Gatherings,USD,nightly,8.5
3058,3207,16740073,111011123,Entire home/apt,"Seattle, WA, United States",2,NaN,12,5.0,5.0,1200,2018-12-20 03:34:43.107959,47.629652,-122.360114,0101000020E61000003B35971B0C975EC035EECD6F98D0...,Beautiful French Style Home with Panoramic Views.,USD,nightly,10.0
2225,2333,13099011,24017652,Private room,"Seattle, WA, United States",0,NaN,2,1.0,1.0,999,2018-12-20 03:07:48.161206,47.671668,-122.357926,0101000020E6100000D7187442E8965EC086E28E37F9D5...,Woodland Park - 9 Cranes Inn,USD,nightly,2.0
2100,2202,12571643,68010866,Private room,"Seattle, WA, United States",10,5.0,2,1.0,1.0,995,2018-12-20 03:29:42.263247,47.651936,-122.352924,0101000020E61000004E7E8B4E96965EC0E38A8BA372D3...,Modern&Luxury Fremont-walk to GOOGL,USD,nightly,2.0
968,1015,5534627,22255793,Entire home/apt,"Bellevue, WA, United States",0,NaN,8,4.0,3.5,995,2018-12-20 03:39:13.309482,47.573161,-122.190667,0101000020E61000004B5B5CE3338C5EC0DC2BF3565DC9...,Otters Cove: Lake Washington Escape,USD,nightly,7.5
7369,7759,30311744,227643634,Private room,"Seattle, WA, United States",1,NaN,8,0.0,4.0,980,2018-12-20 03:20:48.649504,47.637548,-122.312148,0101000020E6100000C1E09A3BFA935EC0ABD0402C9BD1...,1926 15th ave,USD,nightly,4.0


In [14]:
temp_df.tail(10)

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,latitude,longitude,location,name,currency,rate_type,total_rooms
1492,1570,8409941,26967583,Private room,"Seattle, WA, United States",176,4.5,2,1.0,1.0,20,2018-12-20 02:51:53.806173,47.543614,-122.267834,0101000020E610000070B0373124915EC0E9D2BF2495C5...,"Bird's Nest Room, 12 min drive to downtown",USD,nightly,2.0
1491,1569,8409926,26967583,Private room,"Seattle, WA, United States",158,4.5,2,1.0,1.0,20,2018-12-20 02:51:59.122527,47.545520,-122.269886,0101000020E61000007DE9EDCF45915EC02FA86F99D3C5...,"The Family Room, 12 min drive to downtown",USD,nightly,2.0
1233,1295,7095802,36964583,Private room,"Seattle, WA, United States",163,4.5,2,1.0,1.0,19,2018-12-20 02:51:54.88865,47.592136,-122.300080,0101000020E6100000C18BBE8234935EC06364C91CCBCB...,"Quiet Private Room, queen size bed, close to city",USD,nightly,2.0
1351,1421,7693217,2390150,Private room,"Seattle, WA, United States",25,5.0,1,1.0,1.0,19,2018-12-20 02:52:03.230391,47.552515,-122.293456,0101000020E610000029B4ACFBC7925EC054C6BFCFB8C6...,Sweet room with an urban forest feel,USD,nightly,2.0
7336,7726,30181350,76544093,Shared room,"Redmond, WA, United States",0,NaN,2,1.0,1.0,18,2018-12-20 02:53:30.984624,47.667081,-122.140038,0101000020E6100000A08CF161F6885EC0396403E962D5...,A simple space for your stay,USD,nightly,2.0
6184,6478,26665671,26967583,Shared room,"Seattle, WA, United States",6,3.5,2,1.0,1.0,18,2018-12-20 03:09:52.249366,47.642550,-122.302227,0101000020E6100000F73DEAAF57935EC0BC0512143FD2...,The UW Room - Shared Coed (Left Bed),USD,nightly,2.0
4970,5207,22578495,33054882,Private room,"Seattle, WA, United States",71,5.0,2,1.0,1.5,18,2018-12-20 02:51:47.710199,47.588698,-122.307949,0101000020E61000007FF5B86FB5935EC03D9CC0745ACB...,Modern&New - West Room,USD,nightly,2.5
7214,7601,29804289,154167958,Private room,"Bellevue, WA, United States",6,5.0,2,1.0,1.0,17,2018-12-20 02:53:15.841618,47.622748,-122.117583,0101000020E610000032E4D87A86875EC020D3DA34B6CF...,Bellevue Home Near Lake Sammamish 15min to Sea...,USD,nightly,2.0
3244,3403,17618287,24501939,Private room,"Seattle, WA, United States",27,4.5,2,1.0,1.5,16,2018-12-20 04:10:58.288301,47.510071,-122.355852,0101000020E6100000D68D7747C6965EC0ABD1AB014AC1...,Modern W Seattle PVT BD SHR BR | SEA & DWTN 20...,USD,nightly,2.5
7499,7894,30718402,26189599,Shared room,"Seattle, WA, United States",0,NaN,2,1.0,1.0,15,2018-12-20 03:26:57.357244,47.615865,-122.349615,0101000020E61000003ACC971760965EC022E010AAD4CE...,Steps to Pike Place & Space Needle (Female Only),USD,nightly,2.0
